In [ ]:
!pip install kiteconnect

In [2]:
from kiteconnect import KiteConnect

# logging.basicConfig(level=logging.DEBUG)
print('here')
kite = KiteConnect(api_key="aygbpvrrcsaybucb")

# Step 1 - Open the below URL and login and collect the request token
# https://kite.zerodha.com/connect/login?v=4&api_key=aygbpvrrcsaybucb
# Step 2 - Run this program and provide the request token
# Step 3 - You would get access token
# Step 4 - Update the access token in KiteSettings.py

# request_token = input("Request Token: ")
# data = kite.generate_session(request_token, api_secret="ant7m5o74v4x0sptg18gx18kzbkl6i5x")
# kite.set_access_token(data["access_token"])
kite.set_access_token("dwICUtJsEQT1OOQQ0nNkc0TxcboFU2Rb")

# print("====================")
# print("Access Token: ",data["access_token"])

here


In [3]:
import pandas as pd
nse = kite.instruments('NSE')
nse_data = pd.DataFrame(nse)
nse_data

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,256265,1001,NIFTY 50,NIFTY 50,0.0,,0.0,0.00,0,EQ,INDICES,NSE
1,256777,1003,NIFTY MIDCAP 100,NIFTY MIDCAP 100,0.0,,0.0,0.00,0,EQ,INDICES,NSE
2,260105,1016,NIFTY BANK,NIFTY BANK,0.0,,0.0,0.00,0,EQ,INDICES,NSE
3,260617,1018,NIFTY 100,NIFTY 100,0.0,,0.0,0.00,0,EQ,INDICES,NSE
4,257033,1004,NIFTY DIV OPPS 50,NIFTY DIV OPPS 50,0.0,,0.0,0.00,0,EQ,INDICES,NSE
...,...,...,...,...,...,...,...,...,...,...,...,...
8035,194190337,758556,LTFL28-N0,,0.0,,0.0,0.01,1,EQ,NSE,NSE
8036,194190849,758558,KALPATARU,KALPATARU,0.0,,0.0,0.05,1,EQ,NSE,NSE
8037,194192129,758563,ELLEN,ELLENBARRIE INDUS GASES L,0.0,,0.0,0.05,1,EQ,NSE,NSE
8038,194194177,758571,SHKSIL-ST,SHRI HARE-KRI SPO IRON L,0.0,,0.0,0.05,2000,EQ,NSE,NSE


In [4]:
import pandas as pd
import numpy as np
import time as Time
from datetime import datetime, timedelta, time
import yfinance as yf
import pytz
import warnings
import sys
from IPython.display import clear_output

In [5]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [6]:
def find_missing_elements(list1, list2):
    missing_elements = [element for element in list1 if element not in list2]
    return missing_elements

def extract_strings(input_list):
    output_list = []
    for input_string in input_list:
        split_string = input_string.split('.')
        output_list.append(split_string[0])
    return output_list

In [7]:
from datetime import datetime, timedelta
import pandas as pd

def calculate_P(df, k, date):
    days = [14, 28, 56, 112, 224, 336, 448, 896]
    P = 0

    for i in range(len(days)):
        mask = (df.index >= (date - timedelta(days=days[i]))) & (df.index <= date)
        data = df.loc[mask]
        value = ((data['high'] - data['open']) * 100 / data['open'] > k).sum()
        P += (2 ** (len(days) - i - 1)) * value

    return P


def calculate_R(df, k, date):
    days = [14, 28, 56, 112, 224, 336, 448, 896]
    R = 0

    for i in range(len(days)):
        mask = (df.index >= (date - timedelta(days=days[i]))) & (df.index <= date)
        data = df.loc[mask]
        value = ((data['high'] - data['open']) * 100 / data['open'] < k).sum()
        R += (2 ** (len(days) - i - 1)) * value

    return R


def calculate_P2(df, k, date):
    days = [14, 28, 56, 112, 224, 336, 448, 896]
    P = 0

    for i in range(len(days)):
        mask = (df.index >= (date - timedelta(days=days[i]))) & (df.index <= date)
        data = df.loc[mask]
        value = ((data['open'] - data['low']) * 100 / data['open'] > k).sum()
        P += (2 ** (len(days) - i - 1)) * value

    return P


def calculate_R2(df, k, date):
    days = [14, 28, 56, 112, 224, 336, 448, 896]
    R = 0

    for i in range(len(days)):
        mask = (df.index >= (date - timedelta(days=days[i]))) & (df.index <= date)
        data = df.loc[mask]
        value = ((data['open'] - data['low']) * 100 / data['open'] < k).sum()
        R += (2 ** (len(days) - i - 1)) * value

    return R

In [8]:
def get_intraday_margin_sell(symbol):
    order_param_single = [{
            "exchange": "NSE",
            "tradingsymbol": symbol,
            "transaction_type": "SELL",
            "variety": "regular",
            "product": "MIS",
            "order_type": "MARKET",
            "quantity": 1
        }]
    return kite.order_margins(order_param_single)[0]['leverage']

def get_intraday_margin(symbol):

    order_param_single = [{
            "exchange": "NSE",
            "tradingsymbol": symbol,
            "transaction_type": "BUY",
            "variety": "regular",
            "product": "MIS",
            "order_type": "MARKET",
            "quantity": 1
        }]
    return kite.order_margins(order_param_single)[0]['leverage']

def filter_by_margin(input_list, type):
    if(type == 'buy'):
        output = [element for element in input_list if get_intraday_margin(element)==5]
    if(type == 'sell'):
        output = [element for element in input_list if get_intraday_margin_sell(element)==5]
    return output

In [9]:
from datetime import datetime, timedelta
import pandas as pd

def archive_stock_data(symbol, stock_data, date):
    # print(date)
    # stock_data.head()
    try:
        # Define the timeframes
        timeframes = {"2y": "2y", "1y": "1y", "6m": "6mo", "16w": "16wk", "8w": "8wk", "4w": "4wk", "2w": "2wk", "1w": "1wk"}

        # Calculate the P values for each k
        k_values = [0.25, 0.35, 0.5, 0.75, 1, 1.5, 2, 2.5, 3, 4, 5]
        l_values = [0, 0.1, 0.2, 0.3, 0.5]
        p_values = []
        r_values = []
        p_values2 = []
        r_values2 = []

        for k in k_values:
            p_value = calculate_P(stock_data, k, date)
            p_values.append(p_value)
            p_value2 = calculate_P2(stock_data, k, date)
            p_values2.append(p_value2)

        for l in l_values:
            r_value = calculate_R(stock_data, l, date)
            r_values.append(r_value)
            r_value2 = calculate_R2(stock_data, l, date)
            r_values2.append(r_value2)

        return symbol, stock_data, p_values, r_values, p_values2, r_values2

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return symbol, None, None, None, None, None


In [10]:
def retrieve_stock_data(stock_archive, start_date, end_date):
    # Find the start and end date locations
    strt = None
    end = None
    for i, date in enumerate(stock_archive):
        if date == start_date:
            strt = i
        if date == end_date:
            end = i
            break

    # Check if both start and end date locations are found
    if strt is None or end is None:
        print("Start or end date not found in the stock archive.")
        return None

    # Retrieve the sublist from stock_archive
    sublist = stock_archive[strt:end+1]

    return sublist

In [ ]:
column_names = ["Code"]
nifty500 = pd.read_csv("/content/ind_nifty500list.csv", names=column_names)
# nifty500 = pd.read_excel("/content/temp_testing.xlsx", names=column_names)

symbol = nifty500.Code.to_list()

columns = ['Symbol'] + [f'P{k:.2f}' for k in [0.25, 0.35, 0.5, 0.75, 1, 1.5, 2, 2.5, 3, 4, 5]] + [f'R{k:.2f}' for k in [0, 0.1, 0.2, 0.3, 0.5]]

output_df = pd.DataFrame(columns=columns)
output_df2 = pd.DataFrame(columns=columns)

k_values = [0.25, 0.35, 0.5, 0.75, 1, 1.5, 2, 2.5, 3, 4, 5]
l_values = [0, 0.1, 0.2, 0.3, 0.5]
start_date = datetime(2023, 7, 3)
end_date = datetime(2023, 7, 5)
date_range = pd.date_range(start=start_date, end=end_date)
results=[]

stock_archive = {}
diff = (end_date - start_date).days
# print(diff)
for i in symbol[:3]:
    print("== downloading for symbol:   ", i);
    # a = nse_data[nse_data.tradingsymbol == i].instrument_token.values
    a = nse_data[nse_data.tradingsymbol == i[:-3]].instrument_token.values
    try:
      token=a[0]
    except Exception as e:
      print(e)
      continue
    # df=kite.historical_data(token, str(end_date - timedelta(days=365 * 2 + (end_date - start_date).days)), str(end_date), "day", False, False)
    # df=kite.historical_data(7929, str(end_date - timedelta(days=365 * 2 + (end_date - start_date).days)), str(end_date), "day", False, False)
    df=kite.historical_data(token, str(start_date - timedelta(days=365 * 2)), str(end_date), "day", False, False)
    df = pd.DataFrame(df)
    if(len(df)==0):
      print("no data")
      continue
    df['date'] = pd.to_datetime(df['date']).dt.date
    df = df.set_index('date')
    # print(df)
    stock_archive[i] = df

for date_i in range(diff):
    symbol = nifty500.Code.to_list()
    columns = ['Symbol'] + [f'P{k:.2f}' for k in [0.25, 0.35, 0.5, 0.75, 1, 1.5, 2, 2.5, 3, 4, 5]] + [f'R{k:.2f}' for k in [0, 0.1, 0.2, 0.3, 0.5]]
    output_df = pd.DataFrame(columns=columns)
    output_df2 = pd.DataFrame(columns=columns)
    k_values = [0.25, 0.35, 0.5, 0.75, 1, 1.5, 2, 2.5, 3, 4, 5]
    l_values = [0, 0.1, 0.2, 0.3, 0.5]


    for i in symbol[:3]:
        strt = date_i
        strt_date=(start_date+timedelta(days=strt)).strftime('%Y-%m-%d')
        strt_date = datetime.strptime(strt_date, '%Y-%m-%d').date()
        # print(strt_date)
        try:
          end = len(stock_archive[i]) - (end_date - start_date).days + date_i
          # end = date_i
          # end = (stock_archive['days'][-1]-stock_archive['days'][0]).days - (end_date - start_date).days + date_i
          # end_dat=(start_date+timedelta(days=end)).strftime('%Y-%m-%d')
          # print(end_dat)
        except Exception as e:
          continue
        date = start_date + timedelta(days=date_i)
        # print(date)

        # symbol, stock_data, p_values, r_values, p_values2, r_values2 = archive_stock_data(i, stock_archive[i][strt:end], date.date())
        # symbol, stock_data, p_values, r_values, p_values2, r_values2 = archive_stock_data(i, stock_archive[i][stock_archive[i]['date'].find(start_date):stock_archive[i]['date'].find(end_date)], date.date())
        # symbol, stock_data, p_values, r_values, p_values2, r_values2 = archive_stock_data(i, stock_archive[i].loc[(stock_archive[i].index >= strt_date - pd.DateOffset(years=2)) & (stock_archive[i].index < strt_date)], date.date())
        # stock_archive[i].loc[(stock_archive[i]['date'] >= strt_date - pd.DateOffset(years=2)) & (stock_archive[i]['date'] < strt_date)]
        # selected_data = stock_archive[i].loc[(stock_archive[i].index >= strt_date - pd.DateOffset(years=2)) & (stock_archive[i].index < strt_date)]
        symbol, stock_data, p_values, r_values, p_values2, r_values2 = archive_stock_data(i, stock_archive[i].loc[(stock_archive[i].index >= pd.Timestamp(strt_date) - pd.DateOffset(years=2)) & (stock_archive[i].index < pd.Timestamp(strt_date))], pd.Timestamp(strt_date).date())
        print(stock_archive[i].loc[(stock_archive[i].index >= pd.Timestamp(strt_date) - pd.DateOffset(years=2)) & (stock_archive[i].index < pd.Timestamp(strt_date))])
        # print(symbol)
        # print(stock_data)

        if stock_data is None:
            continue

        output_data = {'Symbol': symbol}
        output_data2 = {'Symbol': symbol}

        for i, p_value in enumerate(p_values):
            output_data[f'P{k_values[i]:.2f}'] = p_value

        for i, p_value in enumerate(p_values2):
            output_data2[f'P{k_values[i]:.2f}'] = p_value

        for i, r_value in enumerate(r_values):
            output_data[f'R{l_values[i]:.2f}'] = r_value

        for i, r_value in enumerate(r_values2):
            output_data2[f'R{l_values[i]:.2f}'] = r_value

        output_df.loc[len(output_df)] = output_data
        output_df2.loc[len(output_df2)] = output_data2

    output_df['Pro'] = 8 * output_df['P2.50'] + 4 * output_df['P3.00'] + 4 * output_df['P1.00'] + 8 * output_df['P5.00']
    output_df['Ris'] = 2 * output_df['R0.50'] + 4 * output_df['R0.30'] + 8 * output_df['R0.10']
    output_df['Val'] = output_df['Pro'] * output_df['Pro'] / (output_df['Ris'] + 1)

    output_df2['Pro'] = 8 * output_df2['P2.50'] + 4 * output_df2['P3.00'] + 4 * output_df2['P1.00'] + 8 * output_df2['P5.00']
    output_df2['Ris'] = 2 * output_df2['R0.50'] + 4 * output_df2['R0.30'] + 8 * output_df2['R0.10']
    output_df2['Val'] = output_df2['Pro'] * output_df2['Pro'] / (output_df2['Ris'] + 1)

    df = output_df.sort_values(output_df.columns[-1], ascending=False)
    df_dash = output_df2.sort_values(output_df2.columns[-1], ascending=False)

    output_df['Val2'] = output_df['Pro'] / (output_df['Ris'] * output_df['Ris'] + 1)
    output_df2['Val2'] = output_df2['Pro'] / (output_df2['Ris'] * output_df2['Ris'] + 1)
    # print(output_df)
    # print(output_df2)

    df2 = output_df.sort_values(output_df.columns[-1], ascending=False)
    df2_dash = output_df2.sort_values(output_df2.columns[-1], ascending=False)

    n = 10  # Assumed no. of stocks to be invested
    symbol1 = df2.iloc[:n, 0].tolist()
    symbol2 = df2_dash.iloc[:n, 0].tolist()


    buy_symbols = find_missing_elements(symbol1, symbol2)
    sell_symbols = find_missing_elements(symbol2, symbol1)
    buy_symbols = extract_strings(buy_symbols)
    sell_symbols = extract_strings(sell_symbols)
    BUY_SYMBOLS = filter_by_margin(buy_symbols, 'buy')
    SELL_SYMBOLS = filter_by_margin(sell_symbols, 'sell')

    new_df = {'Date': date.strftime('%Y-%m-%d'), 'Buy': BUY_SYMBOLS[:3], 'SeLl': SELL_SYMBOLS[:3]}

    results.append(new_df)
    print(date.strftime('%Y-%m-%d'))
# Create a dataframe from the results list
out = pd.DataFrame(results)
out

In [ ]:
out.to_csv('/content/nifty200_3.csv', index=False)